Importing the clickbait and non-clickbait sets:

In [1]:
import pickle


clickbait_df = pickle.load(open("clickbait-df", "rb"))
non_clickbait_df = pickle.load(open("non-clickbait-df", "rb"))

In [2]:
clickbait_df.shape

(18317, 11)

In [3]:
non_clickbait_df.shape

(19080, 11)

Defining a custom tokenizer:

In [4]:
import re
import emoji
from gensim.parsing.preprocessing import *


def tokenize(string):

    """ Tokenizes a string.

    Adds a space between numbers and letters, removes punctuation, repeated whitespaces, words shorter than 2
    characters, and stop-words. Returns a list of stems and, eventually, emojis.

    @param string: String to tokenize.
    @return: A list of stems and emojis.
    """

    # Based on the Ranks NL (Google) stopwords list, but "how" and "will" are not stripped, and words shorter than 2
    # characters are not checked (since they are stripped):
    stop_words = [
        "about", "an", "are", "as", "at", "be", "by", "com", "for", "from", "in", "is", "it", "of", "on", "or", "that",
        "the", "this", "to", "was", "what", "when", "where", "who", "with", "the", "www"
    ]

    string = strip_short(
        strip_multiple_whitespaces(
            strip_punctuation(
                split_alphanum(string))),
        minsize=2)
    # Parse emojis:
    emojis = [ c for c in string if c in emoji.UNICODE_EMOJI ]
    # Remove every non-word character and stem each word:
    string = stem_text(re.sub(r"[^\w\s,]", "", string))
    # List of stems and emojis:
    tokens = string.split() + emojis
    
    for stop_word in stop_words:
        try:
            tokens.remove(stop_word)
        except:
            pass

    return tokens

Add a column containing the tokenized titles:

In [5]:
clickbait_df["video_title_tokenized"] = clickbait_df["video_title"].apply(tokenize)
non_clickbait_df["video_title_tokenized"] = non_clickbait_df["video_title"].apply(tokenize)

Splitting the data into train and test set:

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split


clickbait_df["label"] = 1
non_clickbait_df["label"] = 0

# Consider the same number of samples (18000) for each class:
clickbait_df = clickbait_df.sample(frac=1).sample(frac=1).sample(n=18000)
non_clickbait_df = non_clickbait_df.sample(frac=1).sample(frac=1).sample(n=18000)

# Build the dataframe:
dataframe = pd.concat([ clickbait_df, non_clickbait_df ]).sample(frac=1).sample(frac=1)

X_train, X_test, y_train, y_test = train_test_split(
    dataframe.loc[:, dataframe.columns != "label"], 
    dataframe["label"], 
    test_size=0.2, 
    random_state=42)

# Export them:
pickle.dump(X_train, open("x-train", "wb"))
pickle.dump(y_train, open("y-train", "wb"))
pickle.dump(X_test, open("x-test", "wb"))
pickle.dump(y_test, open("y-test", "wb"))

In [7]:
X_train.shape

(28800, 12)

In [8]:
X_test.shape

(7200, 12)

Defining an embedding function:

In [9]:
import numpy as np


def average_embedding(tokens, word2vec, na_vector=None):

    """ Embeds a title with the average representation of its tokens.

    Returns the mean vector representation of the tokens representations. When no token is in the Word2Vec model, it
    can be provided a vector to use instead (for example the mean vector representation of the train set titles).

    @param tokens: List of tokens to embed.
    @param word2vec: Word2Vec model.
    @param na_vector: Vector representation to use when no token is in the Word2Vec model.
    @return: A vector representation for the token list.
    """

    vectors = list()

    for token in tokens:
        if token in word2vec:
            vectors.append(word2vec[token])

    if len(vectors) == 0 and na_vector is not None:
        vectors.append(na_vector)

    return np.mean(np.array(vectors), axis=0)

Training a Word2Vec model onto the train set:

In [10]:
import gensim


documents = X_train["video_title_tokenized"]
word2vec = gensim.models.Word2Vec(
    documents,
    size=25,
    window=20,
    min_count=1,
    workers=2
)
word2vec.train(documents, total_examples=len(documents), epochs=30)

# Export it:
pickle.dump(word2vec, open("word2vec", "wb"))

Get the titles embeddings:

In [11]:
titles_embeddings = X_train["video_title_tokenized"].apply(average_embedding, word2vec=word2vec)
train_set = pd.concat(
    [
        X_train[["video_views", "video_likes", "video_dislikes", "video_comments"]],
        titles_embeddings.apply(pd.Series)
    ], axis=1)
# Add the label column:
train_set["label"] = y_train
# Drop rows with missing values:
train_set = train_set.dropna()

# Compute the average vector representation on the train set, and export it:
mean_title_embedding = titles_embeddings.dropna().mean(axis=0)
pickle.dump(mean_title_embedding, open("mean-title-embedding", "wb"))

# For the test set use the mean title embedding computed on the train set:
titles_embeddings = X_test["video_title_tokenized"].apply(average_embedding, word2vec=word2vec, na_vector=mean_title_embedding)
test_set = pd.concat(
    [
        X_test[["video_views", "video_likes", "video_dislikes", "video_comments"]],
        titles_embeddings.apply(pd.Series)
    ], axis=1)
test_set["label"] = y_test

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)


In [12]:
train_set.shape

(28798, 30)

In [13]:
test_set.shape

(7200, 30)

Considering the logarithm of the video metadata (views, likes, dislikes, comments) to have a normal distribution of values:

In [14]:
# Compute the logarithm of the video metadata (likes, dislikes, comments, views)
train_set[["video_views", "video_likes", "video_dislikes", "video_comments"]] = train_set[["video_views", "video_likes", "video_dislikes", "video_comments"]].apply(np.log)
test_set[["video_views", "video_likes", "video_dislikes", "video_comments"]] = test_set[["video_views", "video_likes", "video_dislikes", "video_comments"]].apply(np.log)
# Replace any -Inf value with 0:
train_set = train_set.replace(-np.inf, 0)
test_set = test_set.replace(-np.inf, 0)

In [15]:
# Remove the label columns:
train_labels = train_set["label"]
test_labels = test_set["label"]
train_set = train_set.drop(columns=["label"])
test_set = test_set.drop(columns=["label"])

# Export the mean values of the metadata in the train set:
pickle.dump(train_set["video_views"].mean(), open("mean-log-video-views", "wb"))
pickle.dump(train_set["video_likes"].mean(), open("mean-log-video-likes", "wb"))
pickle.dump(train_set["video_dislikes"].mean(), open("mean-log-video-dislikes", "wb"))
pickle.dump(train_set["video_comments"].mean(), open("mean-log-video-comments", "wb"))

Train a min-max scaler onto the train set and apply it onto both sets:

In [16]:
from sklearn import preprocessing


min_max_scaler = preprocessing.MinMaxScaler()
min_max_scaler.fit(train_set)
train_set = pd.DataFrame(min_max_scaler.transform(train_set), columns=train_set.columns)
test_set = pd.DataFrame(min_max_scaler.transform(test_set), columns=test_set.columns)
# Export it:
pickle.dump(min_max_scaler, open("min-max-scaler", "wb"))

Train a SVM model:

In [17]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score


svm_params = [
    { "C": np.linspace(1, 25, 10), "gamma": np.linspace(1, 5, 10) },
]

grid_search_cv = GridSearchCV(estimator=SVC(kernel="rbf"), param_grid=svm_params, n_jobs=2, scoring="f1", verbose=3)
grid_search_cv.fit(train_set, train_labels)
predictions = grid_search_cv.predict(test_set)

print("Best SVM with:")
print("\tC:", grid_search_cv.best_params_["C"])
print("\tgamma:", grid_search_cv.best_params_["gamma"])
print("\tBest Score (F1):", grid_search_cv.best_score_)
print("Performance on the test set (%d samples):" % len(test_set))
print("\tAccuracy Score:", accuracy_score(test_labels, predictions))
print("\tArea under ROC curve:", roc_auc_score(test_labels, predictions))
print("\tClassification report (on the test set):")
print(classification_report(test_labels, predictions))

# Export the best estimator:
pickle.dump(grid_search_cv.best_estimator_, open("svm", "wb"))

Fitting 3 folds for each of 100 candidates, totalling 300 fits
[CV] C=1.0, gamma=1.0 ................................................
[CV] C=1.0, gamma=1.0 ................................................
[CV] ....... C=1.0, gamma=1.0, score=0.9601746906519705, total=   5.9s
[CV] C=1.0, gamma=1.0 ................................................
[CV] ....... C=1.0, gamma=1.0, score=0.9587189352188832, total=   5.9s
[CV] C=1.0, gamma=1.4444444444444444 .................................
[CV]  C=1.0, gamma=1.4444444444444444, score=0.9628322259136214, total=   5.6s
[CV] C=1.0, gamma=1.4444444444444444 .................................
[CV] ....... C=1.0, gamma=1.0, score=0.9585195966316666, total=   5.8s
[CV] C=1.0, gamma=1.4444444444444444 .................................
[CV]  C=1.0, gamma=1.4444444444444444, score=0.960990325600749, total=   5.7s
[CV] C=1.0, gamma=1.8888888888888888 .................................
[CV]  C=1.0, gamma=1.4444444444444444, score=0.9603158113442759, total

[Parallel(n_jobs=2)]: Done  28 tasks      | elapsed:  1.9min


[CV] ........ C=1.0, gamma=5.0, score=0.965962319142292, total=   7.2s
[CV] C=3.6666666666666665, gamma=1.0 .................................
[CV] ....... C=1.0, gamma=5.0, score=0.9682655290812611, total=   7.1s
[CV] C=3.6666666666666665, gamma=1.0 .................................
[CV]  C=3.6666666666666665, gamma=1.0, score=0.9639555417056197, total=   5.4s
[CV] C=3.6666666666666665, gamma=1.0 .................................
[CV]  C=3.6666666666666665, gamma=1.0, score=0.9628473306275368, total=   5.3s
[CV] C=3.6666666666666665, gamma=1.4444444444444444 ..................
[CV]  C=3.6666666666666665, gamma=1.0, score=0.9627400103788272, total=   5.4s
[CV] C=3.6666666666666665, gamma=1.4444444444444444 ..................
[CV]  C=3.6666666666666665, gamma=1.4444444444444444, score=0.9659609796596098, total=   5.3s
[CV] C=3.6666666666666665, gamma=1.4444444444444444 ..................
[CV]  C=3.6666666666666665, gamma=1.4444444444444444, score=0.964527202746281, total=   5.3s
[CV] C=3

[CV] C=6.333333333333333, gamma=4.111111111111111 ....................
[CV]  C=6.333333333333333, gamma=3.6666666666666665, score=0.9686882346822012, total=   6.2s
[CV] C=6.333333333333333, gamma=4.111111111111111 ....................
[CV]  C=6.333333333333333, gamma=4.111111111111111, score=0.9682242990654205, total=   6.5s
[CV] C=6.333333333333333, gamma=4.111111111111111 ....................
[CV]  C=6.333333333333333, gamma=4.111111111111111, score=0.9669963560645497, total=   6.5s
[CV] C=6.333333333333333, gamma=4.555555555555555 ....................
[CV]  C=6.333333333333333, gamma=4.111111111111111, score=0.9683465222823824, total=   6.4s
[CV] C=6.333333333333333, gamma=4.555555555555555 ....................
[CV]  C=6.333333333333333, gamma=4.555555555555555, score=0.9675945159950147, total=   7.0s
[CV] C=6.333333333333333, gamma=4.555555555555555 ....................
[CV]  C=6.333333333333333, gamma=4.555555555555555, score=0.9677285030189465, total=   6.7s
[CV] C=6.333333333333

[Parallel(n_jobs=2)]: Done 124 tasks      | elapsed:  8.1min


...

[CV]  C=25.0, gamma=2.333333333333333, score=0.9653878231859883, total=   6.1s
[CV] C=25.0, gamma=2.7777777777777777 ................................
[CV]  C=25.0, gamma=2.333333333333333, score=0.967741935483871, total=   6.5s
[CV] C=25.0, gamma=2.7777777777777777 ................................
[CV]  C=25.0, gamma=2.7777777777777777, score=0.9657534246575343, total=   6.8s
[CV] C=25.0, gamma=2.7777777777777777 ................................
[CV]  C=25.0, gamma=2.7777777777777777, score=0.9648994896364961, total=   6.4s
[CV] C=25.0, gamma=3.2222222222222223 ................................


[Parallel(n_jobs=2)]: Done 284 tasks      | elapsed: 20.0min


[CV]  C=25.0, gamma=2.7777777777777777, score=0.96617050067659, total=   6.5s
[CV] C=25.0, gamma=3.2222222222222223 ................................
[CV]  C=25.0, gamma=3.2222222222222223, score=0.9655530192986097, total=   7.2s
[CV] C=25.0, gamma=3.2222222222222223 ................................
[CV]  C=25.0, gamma=3.2222222222222223, score=0.9649927068139196, total=   6.8s
[CV] C=25.0, gamma=3.6666666666666665 ................................
[CV]  C=25.0, gamma=3.2222222222222223, score=0.9654742096505823, total=   6.9s
[CV] C=25.0, gamma=3.6666666666666665 ................................
[CV]  C=25.0, gamma=3.6666666666666665, score=0.9647376063057457, total=   7.0s
[CV] C=25.0, gamma=3.6666666666666665 ................................
[CV]  C=25.0, gamma=3.6666666666666665, score=0.9650000000000001, total=   7.0s
[CV] C=25.0, gamma=4.111111111111111 .................................
[CV]  C=25.0, gamma=3.6666666666666665, score=0.9647792207792207, total=   6.8s
[CV] C=25.0, gam

[Parallel(n_jobs=2)]: Done 300 out of 300 | elapsed: 21.3min finished


Best SVM with:
	C: 3.6666666666666665
	gamma: 4.111111111111111
	Best Score (F1): 0.9691136727049655
Performance on the test set (7200 samples):
	Accuracy Score: 0.9676388888888889
	Area under ROC curve: 0.9676185993394858
	Classification report (on the test set):
             precision    recall  f1-score   support

          0       0.96      0.97      0.97      3620
          1       0.97      0.96      0.97      3580

avg / total       0.97      0.97      0.97      7200

